In [62]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [63]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from common.extractor.alpaca_extractor import AlpacaExtractor
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio import Portfolio
import numpy as np
import matplotlib.pyplot as plt

In [64]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [65]:
orivault = ADatabase("ori")
alp = AlpacaExtractor(paper=False)

In [66]:
orivault.connect()
portfolio_dictionary = orivault.retrieve("results").to_dict("records")[0]
orivault.disconnect()

In [67]:
pm = Portfolio.from_dict(portfolio_dictionary)

In [68]:
## Retrieve existing stocks 
index = alp.assets().sort_values("ticker")
tickers = index["ticker"].values
tickers_per_batch = int(1000)
batchs = [tickers[i:i + tickers_per_batch] for i in range(0, len(tickers), tickers_per_batch)]
bars = []
[bars.append(alp.latest_bars_bulk(batch)) for batch in batchs]
todays_bar = pd.concat(bars)
index = index.merge(todays_bar,on="ticker",how="left")

'SVA'


In [69]:
end = alp.clock()["date"] - timedelta(days=1)
start = (end - timedelta(days=pm.rolling_window*3))
print(start,end)

2025-04-24 13:53:43.428403-04:00 2025-05-24 13:53:43.428403-04:00


In [70]:
tickers_per_batch = int(10000/(pm.rolling_window*4)/5)
print(tickers_per_batch)

50


In [71]:
relevant_tickers = index[(index["adjclose"]>=pm.min_price) & (index["adjclose"]<=pm.max_price)]["ticker"]
batchs = [relevant_tickers[i:i + tickers_per_batch] for i in range(0, len(relevant_tickers), tickers_per_batch)]

In [80]:
prices = []
for batch in tqdm(batchs):
    try:
        tickers_data = alp.prices_bulk(batch,start,end)
        for ticker in batch:
            try:
                price = tickers_data[tickers_data["ticker"] == ticker].copy()
                bar = todays_bar[todays_bar["ticker"]==ticker][["date","ticker","adjclose","high","low","volume"]].copy()
                price = p.lower_column(price)
                price = pd.concat([price,bar])
                price = p.utc_date(price)
                price.sort_values("date", inplace=True)
                price = p.additional_date_columns(price)
                price["market_cap"] = price["adjclose"] * price["volume"]
                if price[(price["adjclose"]<=pm.max_price) & (price["adjclose"]>=pm.min_price)].index.size > 0:
                    price["standard_dev"] = 1
                    price["coefficient_of_variation"] = 1
                    for member in Indicator:
                        price = member.calculate(price,timeframe=pm.rolling_window,live=True)
                    prices.append(price)
            except Exception as e:
                print(str(e))
    except Exception as e:
        print(str(e))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.07s/it]


In [82]:
simulation = pd.concat(prices)
simulation.sort_values("date", inplace=True)
trades = pm.recs(simulation.copy())
trades

In [84]:
orivault.connect()
orivault.drop("recommendations")
orivault.store("recommendations",trades)
orivault.disconnect()